In [1]:
import tupak
import matplotlib.pyplot as plt
import numpy as np

10:55 tupak INFO    : Running tupak version: 0.2.2: (UNCLEAN) 3b48fec 2018-09-02 21:49:04 +1000


In [ ]:

"""
Tutorial to demonstrate running parameter estimation on a reduced parameter space for an injected signal.
This example estimates the masses using a uniform prior in both component masses and distance using a uniform in
comoving volume prior on luminosity distance between luminosity distances of 100Mpc and 5Gpc, the cosmology is WMAP7.
"""
from __future__ import division, print_function
import numpy as np
import tupak
# Set the duration and sampling frequency of the data segment that we're going to inject the signal into
duration = 4.
sampling_frequency = 2048.
# Specify the output directory and the name of the simulation.
outdir = 'outdir'
label = 'basic_tutorial'
tupak.core.utils.setup_logger(outdir=outdir, label=label)
# Set up a random seed for result reproducibility.  This is optional!
np.random.seed(88170235)
# We are going to inject a binary black hole waveform.  We first establish a dictionary of parameters that
# includes all of the different waveform parameters, including masses of the two black holes (mass_1, mass_2),
# spins of both black holes (a, tilt, phi), etc.
injection_parameters = dict(mass_1=36., mass_2=29., a_1=0.4, a_2=0.3, tilt_1=0.5, tilt_2=1.0, phi_12=1.7, phi_jl=0.3,
                            luminosity_distance=2000., iota=0.4, psi=2.659, phase=1.3, geocent_time=1126259642.413,
                            ra=1.375, dec=-1.2108)
#injection_parameters = dict(mass_1=36., mass_2=29., a_1=0.4, a_2=0.3, tilt_1=0.5, tilt_2=1.0, phi_12=1.7, phi_jl=0.3,
                            #luminosity_distance=1000., iota=0.4, psi=2.659, phase=1.3, geocent_time=1126259642.413,
                            #ra=1.375, dec=-1.2108)
# Fixed arguments passed into the source model
waveform_arguments = dict(waveform_approximant='IMRPhenomPv2',
                          reference_frequency=50.)
# Create the waveform_generator using a LAL BinaryBlackHole source function
waveform_generator = tupak.gw.WaveformGenerator(
    duration=duration, sampling_frequency=sampling_frequency,
    frequency_domain_source_model=tupak.gw.source.lal_binary_black_hole,
    parameters=injection_parameters, waveform_arguments=waveform_arguments)
hf_signal = waveform_generator.frequency_domain_strain()
# Set up interferometers.  In this case we'll use two interferometers
# (LIGO-Hanford (H1), LIGO-Livingston (L1). These default to their design
# sensitivity
IFOs = []
for name in ["H1", "L1","V1"]: ##V1 virgo, ET Einstein Telescope, #K1 Kagra
    IFOs.append(
        tupak.gw.detector.get_interferometer_with_fake_noise_and_injection(
            name, injection_polarizations=hf_signal,
            injection_parameters=injection_parameters, duration=duration,
            sampling_frequency=sampling_frequency, outdir=outdir))
# Set up prior, which is a dictionary
# By default we will sample all terms in the signal models.  However, this will take a long time for the calculation,
# so for this example we will set almost all of the priors to be equall to their injected values.  This implies the
# prior is a delta function at the true, injected value.  In reality, the sampler implementation is smart enough to
# not sample any parameter that has a delta-function prior.
# The above list does *not* include mass_1, mass_2, iota and luminosity_distance, which means those are the parameters
# that will be included in the sampler.  If we do nothing, then the default priors get used.
priors = tupak.gw.prior.BBHPriorSet()
priors['geocent_time'] = tupak.core.prior.Uniform(
    minimum=injection_parameters['geocent_time'] - 1, maximum=injection_parameters['geocent_time'] + 1,
    name='geocent_time', latex_label='$t_c$')
#for key in ['a_1', 'a_2', 'tilt_1', 'tilt_2', 'phi_12', 'phi_jl', 'psi', 'geocent_time', 'phase']:
    #priors[key] = injection_parameters[key]
# Initialise the likelihood by passing in the interferometer data (IFOs) and the waveoform generator
likelihood = tupak.gw.GravitationalWaveTransient(interferometers=IFOs, waveform_generator=waveform_generator,
                                              time_marginalization=False, phase_marginalization=False,
                                              distance_marginalization=False, prior=priors)
# Run sampler.  In this case we're going to use the `dynesty` sampler
result = tupak.run_sampler(likelihood=likelihood, priors=priors, sampler='dynesty', npoints=1000,nlive=5000,walks=500,dlogz=0.01,resume=True,
                           injection_parameters=injection_parameters, outdir=outdir, label=label)
# make some plots of the outputs
result.plot_corner()


10:56 tupak INFO    : No power spectral density provided, using aLIGO,zero detuning, high power.
/home/brandon/.conda/envs/pycbc/lib/python3.6/site-packages/tupak/gw/detector.py:1746: RuntimeWarning: invalid value encountered in multiply
  frequency_domain_strain = self.power_spectral_density_interpolated(frequencies) ** 0.5 * white_noise
10:56 tupak INFO    : Injected signal in H1:
10:56 tupak INFO    :   optimal SNR = 12.02
10:56 tupak INFO    :   matched filter SNR = 11.70
10:56 tupak INFO    :   mass_1 = 36.0
10:56 tupak INFO    :   mass_2 = 29.0
10:56 tupak INFO    :   a_1 = 0.4
10:56 tupak INFO    :   a_2 = 0.3
10:56 tupak INFO    :   tilt_1 = 0.5
10:56 tupak INFO    :   tilt_2 = 1.0
10:56 tupak INFO    :   phi_12 = 1.7
10:56 tupak INFO    :   phi_jl = 0.3
10:56 tupak INFO    :   luminosity_distance = 2000.0
10:56 tupak INFO    :   iota = 0.4
10:56 tupak INFO    :   psi = 2.659
10:56 tupak INFO    :   phase = 1.3
10:56 tupak INFO    :   geocent_time = 1126259642.413
10:56 tupak I

 52775| logz ratio= 9.207 +/-  0.012 | dlogz: 91.025 >  0.010/home/brandon/.conda/envs/pycbc/lib/python3.6/site-packages/dynesty/bounding.py:1223: UserWarning: Volume is sparsely sampled. MLE covariance estimates and associated ellipsoid decompositions might be unstable.
  warnings.warn("Volume is sparsely sampled. MLE covariance "
/home/brandon/.conda/envs/pycbc/lib/python3.6/site-packages/dynesty/bounding.py:1223: UserWarning: Volume is sparsely sampled. MLE covariance estimates and associated ellipsoid decompositions might be unstable.
  warnings.warn("Volume is sparsely sampled. MLE covariance "
 52781| logz ratio= 9.213 +/-  0.012 | dlogz: 91.017 >  0.010/home/brandon/.conda/envs/pycbc/lib/python3.6/site-packages/dynesty/bounding.py:1223: UserWarning: Volume is sparsely sampled. MLE covariance estimates and associated ellipsoid decompositions might be unstable.
  warnings.warn("Volume is sparsely sampled. MLE covariance "
/home/brandon/.conda/envs/pycbc/lib/python3.6/site-packages

 52859| logz ratio= 9.289 +/-  0.012 | dlogz: 90.926 >  0.010/home/brandon/.conda/envs/pycbc/lib/python3.6/site-packages/dynesty/bounding.py:1223: UserWarning: Volume is sparsely sampled. MLE covariance estimates and associated ellipsoid decompositions might be unstable.
  warnings.warn("Volume is sparsely sampled. MLE covariance "
 52865| logz ratio= 9.294 +/-  0.012 | dlogz: 90.919 >  0.010/home/brandon/.conda/envs/pycbc/lib/python3.6/site-packages/dynesty/bounding.py:1223: UserWarning: Volume is sparsely sampled. MLE covariance estimates and associated ellipsoid decompositions might be unstable.
  warnings.warn("Volume is sparsely sampled. MLE covariance "
/home/brandon/.conda/envs/pycbc/lib/python3.6/site-packages/dynesty/bounding.py:1223: UserWarning: Volume is sparsely sampled. MLE covariance estimates and associated ellipsoid decompositions might be unstable.
  warnings.warn("Volume is sparsely sampled. MLE covariance "
 52871| logz ratio= 9.300 +/-  0.012 | dlogz: 90.912 >  0.0

 52955| logz ratio= 9.381 +/-  0.012 | dlogz: 90.815 >  0.010/home/brandon/.conda/envs/pycbc/lib/python3.6/site-packages/dynesty/bounding.py:1223: UserWarning: Volume is sparsely sampled. MLE covariance estimates and associated ellipsoid decompositions might be unstable.
  warnings.warn("Volume is sparsely sampled. MLE covariance "
 52967| logz ratio= 9.392 +/-  0.012 | dlogz: 90.801 >  0.010/home/brandon/.conda/envs/pycbc/lib/python3.6/site-packages/dynesty/bounding.py:1223: UserWarning: Volume is sparsely sampled. MLE covariance estimates and associated ellipsoid decompositions might be unstable.
  warnings.warn("Volume is sparsely sampled. MLE covariance "
 52973| logz ratio= 9.398 +/-  0.012 | dlogz: 90.794 >  0.010/home/brandon/.conda/envs/pycbc/lib/python3.6/site-packages/dynesty/bounding.py:1223: UserWarning: Volume is sparsely sampled. MLE covariance estimates and associated ellipsoid decompositions might be unstable.
  warnings.warn("Volume is sparsely sampled. MLE covariance 

/home/brandon/.conda/envs/pycbc/lib/python3.6/site-packages/dynesty/bounding.py:1223: UserWarning: Volume is sparsely sampled. MLE covariance estimates and associated ellipsoid decompositions might be unstable.
  warnings.warn("Volume is sparsely sampled. MLE covariance "
/home/brandon/.conda/envs/pycbc/lib/python3.6/site-packages/dynesty/bounding.py:1223: UserWarning: Volume is sparsely sampled. MLE covariance estimates and associated ellipsoid decompositions might be unstable.
  warnings.warn("Volume is sparsely sampled. MLE covariance "
 53081| logz ratio= 9.502 +/-  0.013 | dlogz: 90.669 >  0.010/home/brandon/.conda/envs/pycbc/lib/python3.6/site-packages/dynesty/bounding.py:1223: UserWarning: Volume is sparsely sampled. MLE covariance estimates and associated ellipsoid decompositions might be unstable.
  warnings.warn("Volume is sparsely sampled. MLE covariance "
/home/brandon/.conda/envs/pycbc/lib/python3.6/site-packages/dynesty/bounding.py:1223: UserWarning: Volume is sparsely sa

...

/home/brandon/.conda/envs/pycbc/lib/python3.6/site-packages/dynesty/bounding.py:1223: UserWarning: Volume is sparsely sampled. MLE covariance estimates and associated ellipsoid decompositions might be unstable.
  warnings.warn("Volume is sparsely sampled. MLE covariance "
 60053| logz ratio=16.521 +/-  0.021 | dlogz: 82.255 >  0.010/home/brandon/.conda/envs/pycbc/lib/python3.6/site-packages/dynesty/bounding.py:1223: UserWarning: Volume is sparsely sampled. MLE covariance estimates and associated ellipsoid decompositions might be unstable.
  warnings.warn("Volume is sparsely sampled. MLE covariance "
/home/brandon/.conda/envs/pycbc/lib/python3.6/site-packages/dynesty/bounding.py:1223: UserWarning: Volume is sparsely sampled. MLE covariance estimates and associated ellipsoid decompositions might be unstable.
  warnings.warn("Volume is sparsely sampled. MLE covariance "
/home/brandon/.conda/envs/pycbc/lib/python3.6/site-packages/dynesty/bounding.py:1223: UserWarning: Volume is sparsely sa

 60149| logz ratio=16.616 +/-  0.021 | dlogz: 82.141 >  0.010/home/brandon/.conda/envs/pycbc/lib/python3.6/site-packages/dynesty/bounding.py:1223: UserWarning: Volume is sparsely sampled. MLE covariance estimates and associated ellipsoid decompositions might be unstable.
  warnings.warn("Volume is sparsely sampled. MLE covariance "
 60155| logz ratio=16.622 +/-  0.021 | dlogz: 82.134 >  0.010/home/brandon/.conda/envs/pycbc/lib/python3.6/site-packages/dynesty/bounding.py:1223: UserWarning: Volume is sparsely sampled. MLE covariance estimates and associated ellipsoid decompositions might be unstable.
  warnings.warn("Volume is sparsely sampled. MLE covariance "
 60161| logz ratio=16.628 +/-  0.021 | dlogz: 82.127 >  0.010/home/brandon/.conda/envs/pycbc/lib/python3.6/site-packages/dynesty/bounding.py:1223: UserWarning: Volume is sparsely sampled. MLE covariance estimates and associated ellipsoid decompositions might be unstable.
  warnings.warn("Volume is sparsely sampled. MLE covariance 

 60269| logz ratio=16.737 +/-  0.021 | dlogz: 81.996 >  0.010/home/brandon/.conda/envs/pycbc/lib/python3.6/site-packages/dynesty/bounding.py:1223: UserWarning: Volume is sparsely sampled. MLE covariance estimates and associated ellipsoid decompositions might be unstable.
  warnings.warn("Volume is sparsely sampled. MLE covariance "
/home/brandon/.conda/envs/pycbc/lib/python3.6/site-packages/dynesty/bounding.py:1223: UserWarning: Volume is sparsely sampled. MLE covariance estimates and associated ellipsoid decompositions might be unstable.
  warnings.warn("Volume is sparsely sampled. MLE covariance "
 60275| logz ratio=16.743 +/-  0.021 | dlogz: 81.989 >  0.010/home/brandon/.conda/envs/pycbc/lib/python3.6/site-packages/dynesty/bounding.py:1223: UserWarning: Volume is sparsely sampled. MLE covariance estimates and associated ellipsoid decompositions might be unstable.
  warnings.warn("Volume is sparsely sampled. MLE covariance "
/home/brandon/.conda/envs/pycbc/lib/python3.6/site-packages

 60359| logz ratio=16.828 +/-  0.021 | dlogz: 81.888 >  0.010/home/brandon/.conda/envs/pycbc/lib/python3.6/site-packages/dynesty/bounding.py:1223: UserWarning: Volume is sparsely sampled. MLE covariance estimates and associated ellipsoid decompositions might be unstable.
  warnings.warn("Volume is sparsely sampled. MLE covariance "
/home/brandon/.conda/envs/pycbc/lib/python3.6/site-packages/dynesty/bounding.py:1223: UserWarning: Volume is sparsely sampled. MLE covariance estimates and associated ellipsoid decompositions might be unstable.
  warnings.warn("Volume is sparsely sampled. MLE covariance "
/home/brandon/.conda/envs/pycbc/lib/python3.6/site-packages/dynesty/bounding.py:1223: UserWarning: Volume is sparsely sampled. MLE covariance estimates and associated ellipsoid decompositions might be unstable.
  warnings.warn("Volume is sparsely sampled. MLE covariance "
/home/brandon/.conda/envs/pycbc/lib/python3.6/site-packages/dynesty/bounding.py:1223: UserWarning: Volume is sparsely sa

 60431| logz ratio=16.901 +/-  0.021 | dlogz: 81.800 >  0.010/home/brandon/.conda/envs/pycbc/lib/python3.6/site-packages/dynesty/bounding.py:1223: UserWarning: Volume is sparsely sampled. MLE covariance estimates and associated ellipsoid decompositions might be unstable.
  warnings.warn("Volume is sparsely sampled. MLE covariance "
/home/brandon/.conda/envs/pycbc/lib/python3.6/site-packages/dynesty/bounding.py:1223: UserWarning: Volume is sparsely sampled. MLE covariance estimates and associated ellipsoid decompositions might be unstable.
  warnings.warn("Volume is sparsely sampled. MLE covariance "
 60437| logz ratio=16.907 +/-  0.021 | dlogz: 81.793 >  0.010/home/brandon/.conda/envs/pycbc/lib/python3.6/site-packages/dynesty/bounding.py:1223: UserWarning: Volume is sparsely sampled. MLE covariance estimates and associated ellipsoid decompositions might be unstable.
  warnings.warn("Volume is sparsely sampled. MLE covariance "
 60443| logz ratio=16.913 +/-  0.021 | dlogz: 81.786 >  0.0

In [ ]:
result.plot_corner()

In [ ]:
plt.plot(np.power(hf_signal['plus'],2.0)+np.power(hf_signal['cross'],2.0))

In [ ]:
IFOs[0].plot_data()


In [ ]:
tupak.gw.detector.get_interferometer_with_fake_noise_and_injection